In [38]:
%%file quiz1.py
from mrjob.job import MRJob
from mrjob.step import MRStep

class getAdsInfo(MRJob):
    def mapper_init(self):
        self.adInfo={}
     
    def mapper(self,_,line):
        uID,adID,action=line.split(", ")
        self.adInfo.setdefault(adID,[0,0,(0,0)]) #Times Skipped, Times Clicked, Times Watched
        
        if('skip'==action):
            self.adInfo[adID][0]=self.adInfo[adID][0]+1
        elif('click'==action):
            self.adInfo[adID][1]=self.adInfo[adID][1]+1
        elif('watch' in action):
            duration,count=self.adInfo[adID][2]
            newDuration=int(action.split(' ')[1].split('sec')[0])
            self.adInfo[adID][2]=duration+newDuration,count+1
            
    def mapper_final(self):
        for key,value in self.adInfo.items():
            yield key,value
        
            
    def combiner(self,key,value):
        skipCountTotal=0
        clickedCountTotal=0
        watchDurationTotal=0
        watchCountTotal=0
        for skipCount,clickedCount,watchCount in value:
            skipCountTotal=skipCountTotal+skipCount
            clickedCountTotal=clickedCountTotal+clickedCount
            
            
            watchDuration,watchCount=watchCount
            watchDurationTotal=watchDurationTotal+watchDuration
            watchCountTotal=watchCountTotal+watchCount
            
        yield key,(skipCountTotal,clickedCountTotal,(watchDurationTotal,watchCountTotal))
  
        
    def reducer(self,key,value):
        skipCountTotal=0
        clickedCountTotal=0
        watchDurationTotal=0
        watchCountTotal=0
        for skipCount,clickedCount,watchCount in value:
            skipCountTotal=skipCountTotal+skipCount
            clickedCountTotal=clickedCountTotal+clickedCount
            
            
            watchDuration,watchCount=watchCount
            watchDurationTotal=watchDurationTotal+watchDuration
            watchCountTotal=watchCountTotal+watchCount

            
        avgTime=watchDurationTotal/watchCountTotal if watchCountTotal > 0 else 0    
        adInformation="ADD "+str(key)+", skip "+str(skipCountTotal)+', click '+str(clickedCountTotal)+', watch '+str(watchCountTotal)+'; average time it is watched for '+str(avgTime)+' sec'   
        yield "->",adInformation
        
            
    
    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
                  mapper=self.mapper,
                  mapper_final=self.mapper_final,
                  combiner=self.combiner,
                  reducer=self.reducer)
        ]
    
    
if __name__=='__main__':
        getAdsInfo.run()

Overwriting quiz1.py


In [39]:
!python quiz1.py Advertisments.txt

"->"	"ADD 022, skip 1, click 0, watch 2; average time it is watched for 4.0 sec"
"->"	"ADD 234, skip 1, click 2, watch 1; average time it is watched for 2.0 sec"


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\DELL\AppData\Local\Temp\quiz1.DELL.20240531.130734.025289
Running step 1 of 1...
job output is in C:\Users\DELL\AppData\Local\Temp\quiz1.DELL.20240531.130734.025289\output
Streaming final output from C:\Users\DELL\AppData\Local\Temp\quiz1.DELL.20240531.130734.025289\output...
Removing temp directory C:\Users\DELL\AppData\Local\Temp\quiz1.DELL.20240531.130734.025289...


In [52]:
%%file getAVG.py
from mrjob.job import MRJob
from mrjob.step import MRStep

class GetAverage(MRJob):
    def mapper(self, _, line):
        yield None,(int(line),1)

    def combiner(self, key, values):
        sumTotal=0
        countTotal=0
        for sumVal,countVal in values:
            sumTotal=sumTotal+sumVal
            countTotal=countTotal+countVal
            
        yield None,(sumTotal,countTotal)
    def reducer(self, key, values):
        sumTotal=0
        countTotal=0
        for sumVal,countVal in values:
            sumTotal=sumTotal+sumVal
            countTotal=countTotal+countVal       
        yield None,(sumTotal/countTotal)

    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                combiner=self.combiner,
                reducer=self.reducer
            )
        ]

if __name__ == '__main__':
    GetAverage.run()

Overwriting getAVG.py


In [53]:
!python getAVG.py values.txt

null	4.4


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\DELL\AppData\Local\Temp\getAVG.DELL.20240531.134005.766303
Running step 1 of 1...
job output is in C:\Users\DELL\AppData\Local\Temp\getAVG.DELL.20240531.134005.766303\output
Streaming final output from C:\Users\DELL\AppData\Local\Temp\getAVG.DELL.20240531.134005.766303\output...
Removing temp directory C:\Users\DELL\AppData\Local\Temp\getAVG.DELL.20240531.134005.766303...
